# COSMOS: 多机器人编队导航演示

**COSMOS** (COordinated Safety On Manifold for multi-agent Systems) + RMPflow + MAPPO

---

## 1. 环境配置

In [ ]:
# 安装依赖
!pip install gymnasium wandb -q

# 验证环境
import torch
import numpy as np

print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 2. 配置 Keys (GitHub & WandB)

In [ ]:
# @title 配置 API Keys { run: "auto" }
from google.colab import userdata
import os

# 方式1: 从 Colab Secrets 读取 (推荐)
# 在 Colab 左侧 🔑 图标添加 secrets: GITHUB_TOKEN, WANDB_API_KEY
try:
    GITHUB_TOKEN = userdata.get('GITHUB_TOKEN')
    print("✓ GitHub Token 已从 Secrets 加载")
except:
    GITHUB_TOKEN = None
    print("⚠ GitHub Token 未设置 (如果仓库是公开的则不需要)")

try:
    WANDB_API_KEY = userdata.get('WANDB_API_KEY')
    os.environ['WANDB_API_KEY'] = WANDB_API_KEY
    print("✓ WandB API Key 已从 Secrets 加载")
except:
    WANDB_API_KEY = None
    print("⚠ WandB API Key 未设置 (将使用离线模式)")

In [ ]:
# 方式2: 手动输入 (如果 Secrets 未设置)
from getpass import getpass

if GITHUB_TOKEN is None:
    print("请输入 GitHub Token (留空则尝试公开克隆):")
    GITHUB_TOKEN = getpass("GitHub Token: ") or None

if WANDB_API_KEY is None:
    print("请输入 WandB API Key (留空则使用离线模式):")
    WANDB_API_KEY = getpass("WandB API Key: ") or None
    if WANDB_API_KEY:
        os.environ['WANDB_API_KEY'] = WANDB_API_KEY

## 3. 克隆代码库

In [ ]:
import os

REPO_NAME = "safe-rl-manifold-suite"
GITHUB_USERNAME = "ustbmicl-ros2epucksRL"
REPO_URL = f"https://github.com/{GITHUB_USERNAME}/{REPO_NAME}.git"

# 检查是否已克隆
if os.path.exists(f"/content/{REPO_NAME}"):
    print(f"仓库已存在，切换到目录...")
    %cd /content/{REPO_NAME}
    !git pull
else:
    # 克隆仓库
    if GITHUB_TOKEN:
        # 使用 Token 克隆私有仓库
        clone_url = f"https://{GITHUB_TOKEN}@github.com/{GITHUB_USERNAME}/{REPO_NAME}.git"
        !git clone {clone_url}
    else:
        # 尝试公开克隆
        !git clone {REPO_URL}
    
    %cd /content/{REPO_NAME}

print(f"\n当前目录: {os.getcwd()}")
!ls -la

## 4. 导入模块

In [ ]:
import sys
sys.path.insert(0, '.')

import numpy as np
import torch
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from IPython.display import HTML, display
from matplotlib.animation import FuncAnimation
import time

from formation_nav.config import Config
from formation_nav.env.formation_env import FormationNavEnv
from formation_nav.env.formations import FormationTopology
from formation_nav.safety import COSMOS, COSMOSMode
from formation_nav.algo.mappo import MAPPO
from formation_nav.algo.buffer import RolloutBuffer

print("✓ 模块导入成功!")

## 5. 配置参数

In [ ]:
# @title 训练参数 { run: "auto" }

NUM_EPISODES = 100  # @param {type: "slider", min: 20, max: 500, step: 10}
NUM_AGENTS = 4  # @param [3, 4, 5, 6] {type: "raw"}
FORMATION = "square"  # @param ["square", "triangle", "circle", "line", "hexagon"]
SEED = 42  # @param {type: "integer"}
USE_WANDB = False  # @param {type: "boolean"}

print(f"训练轮数:   {NUM_EPISODES}")
print(f"智能体数量: {NUM_AGENTS}")
print(f"编队形状:   {FORMATION}")
print(f"随机种子:   {SEED}")
print(f"使用 WandB: {USE_WANDB}")

In [ ]:
# @title 环境参数 { run: "auto" }

ARENA_SIZE = 5.0  # @param {type: "slider", min: 3.0, max: 10.0, step: 0.5}
NUM_OBSTACLES = 4  # @param {type: "slider", min: 0, max: 8, step: 1}
MAX_STEPS = 500  # @param {type: "slider", min: 100, max: 1000, step: 50}
DT = 0.02  # @param {type: "number"}
FORMATION_SCALE = 1.0  # @param {type: "slider", min: 0.5, max: 2.0, step: 0.1}

print(f"场地大小:   {ARENA_SIZE} x {ARENA_SIZE}")
print(f"障碍物数量: {NUM_OBSTACLES}")
print(f"最大步数:   {MAX_STEPS}")
print(f"时间步长:   {DT}")
print(f"编队缩放:   {FORMATION_SCALE}")

In [ ]:
# @title 安全参数 (COSMOS) { run: "auto" }

SAFETY_RADIUS = 0.5  # @param {type: "slider", min: 0.2, max: 1.0, step: 0.1}
OBSTACLE_MARGIN = 0.3  # @param {type: "slider", min: 0.1, max: 0.5, step: 0.05}
BOUNDARY_MARGIN = 0.3  # @param {type: "slider", min: 0.1, max: 0.5, step: 0.05}
CBF_ALPHA = 1.0  # @param {type: "slider", min: 0.5, max: 3.0, step: 0.1}
COSMOS_MODE = "decentralized"  # @param ["centralized", "decentralized"]

print(f"安全半径:     {SAFETY_RADIUS}")
print(f"障碍物边距:   {OBSTACLE_MARGIN}")
print(f"边界边距:     {BOUNDARY_MARGIN}")
print(f"CBF alpha:    {CBF_ALPHA}")
print(f"COSMOS 模式:  {COSMOS_MODE}")

In [ ]:
# @title 算法参数 (MAPPO) { run: "auto" }

LEARNING_RATE = 3e-4  # @param {type: "number"}
PPO_EPOCHS = 10  # @param {type: "slider", min: 5, max: 20, step: 1}
BATCH_SIZE = 64  # @param [32, 64, 128, 256] {type: "raw"}
GAMMA = 0.99  # @param {type: "slider", min: 0.9, max: 0.999, step: 0.001}
GAE_LAMBDA = 0.95  # @param {type: "slider", min: 0.9, max: 1.0, step: 0.01}
CLIP_EPSILON = 0.2  # @param {type: "slider", min: 0.1, max: 0.3, step: 0.05}
ENTROPY_COEF = 0.01  # @param {type: "number"}

print(f"学习率:       {LEARNING_RATE}")
print(f"PPO 轮数:     {PPO_EPOCHS}")
print(f"批大小:       {BATCH_SIZE}")
print(f"折扣因子:     {GAMMA}")
print(f"GAE lambda:   {GAE_LAMBDA}")
print(f"Clip epsilon: {CLIP_EPSILON}")
print(f"熵系数:       {ENTROPY_COEF}")

In [ ]:
# 创建配置对象 (应用上述所有参数)
np.random.seed(SEED)
torch.manual_seed(SEED)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

cfg = Config()

# 环境配置
cfg.env.num_agents = NUM_AGENTS
cfg.env.formation_shape = FORMATION
cfg.env.arena_size = ARENA_SIZE
cfg.env.num_obstacles = NUM_OBSTACLES
cfg.env.max_steps = MAX_STEPS
cfg.env.dt = DT
cfg.env.formation_scale = FORMATION_SCALE

# 安全配置
cfg.safety.safety_radius = SAFETY_RADIUS
cfg.safety.obstacle_margin = OBSTACLE_MARGIN
cfg.safety.boundary_margin = BOUNDARY_MARGIN
cfg.safety.cbf_alpha = CBF_ALPHA

# 算法配置
cfg.algo.lr = LEARNING_RATE
cfg.algo.ppo_epochs = PPO_EPOCHS
cfg.algo.batch_size = BATCH_SIZE
cfg.algo.gamma = GAMMA
cfg.algo.gae_lambda = GAE_LAMBDA
cfg.algo.clip_epsilon = CLIP_EPSILON
cfg.algo.entropy_coef = ENTROPY_COEF

# 训练配置
cfg.train.seed = SEED

print("=" * 50)
print("配置汇总")
print("=" * 50)
print(f"设备:         {DEVICE}")
print(f"智能体:       {cfg.env.num_agents} x {cfg.env.formation_shape}")
print(f"场地:         {cfg.env.arena_size} x {cfg.env.arena_size}")
print(f"安全半径:     {cfg.safety.safety_radius}")
print(f"COSMOS 模式:  {COSMOS_MODE}")
print("=" * 50)

## 6. 初始化 WandB (可选)

In [ ]:
if USE_WANDB:
    import wandb
    
    wandb.init(
        project="cosmos-formation-nav",
        name=f"cosmos_{FORMATION}_n{NUM_AGENTS}_ep{NUM_EPISODES}",
        config={
            # 训练参数
            "num_episodes": NUM_EPISODES,
            "seed": SEED,
            # 环境参数
            "num_agents": NUM_AGENTS,
            "formation": FORMATION,
            "arena_size": ARENA_SIZE,
            "num_obstacles": NUM_OBSTACLES,
            "max_steps": MAX_STEPS,
            # 安全参数
            "safety_radius": SAFETY_RADIUS,
            "cosmos_mode": COSMOS_MODE,
            "cbf_alpha": CBF_ALPHA,
            # 算法参数
            "learning_rate": LEARNING_RATE,
            "ppo_epochs": PPO_EPOCHS,
            "batch_size": BATCH_SIZE,
            "gamma": GAMMA,
            "device": DEVICE,
        }
    )
    print("✓ WandB 初始化成功")
else:
    print("跳过 WandB (USE_WANDB=False)")

## 7. 创建环境和模型

In [ ]:
# 环境
env = FormationNavEnv(cfg.env, cfg.reward)
topology = FormationTopology(cfg.env.num_agents, "complete")

# COSMOS 模式
cosmos_mode = COSMOSMode.CENTRALIZED if COSMOS_MODE == "centralized" else COSMOSMode.DECENTRALIZED

# COSMOS 安全滤波器
cosmos = COSMOS(
    env_cfg=cfg.env,
    safety_cfg=cfg.safety,
    desired_distances=env.desired_distances,
    topology_edges=topology.edges(),
    obstacle_positions=env.obstacles,
    mode=cosmos_mode
)

# MAPPO
obs_dim = env.observation_space.shape[0]
share_obs_dim = env.share_observation_space.shape[0]

mappo = MAPPO(obs_dim, share_obs_dim, act_dim=2, cfg=cfg.algo, device=DEVICE)

# Buffer
buffer = RolloutBuffer(
    episode_length=cfg.env.max_steps,
    num_agents=cfg.env.num_agents,
    obs_dim=obs_dim,
    share_obs_dim=share_obs_dim,
    act_dim=2,
    gamma=cfg.algo.gamma,
    gae_lambda=cfg.algo.gae_lambda,
    device=DEVICE
)

print(f"观测维度:    {obs_dim}")
print(f"共享观测:    {share_obs_dim}")
print(f"COSMOS 模式: {cosmos_mode.value}")
print(f"设备:        {DEVICE}")
print("\n✓ 初始化完成!")

## 8. 训练

In [ ]:
# 训练记录
metrics = {'episode': [], 'reward': [], 'cost': [], 'formation_error': [], 'min_dist': [], 'collisions': []}

print("开始训练...")
print("=" * 70)

start_time = time.time()
total_steps = 0
LOG_INTERVAL = max(1, NUM_EPISODES // 10)  # 打印10次

for episode in range(NUM_EPISODES):
    obs, share_obs, _ = env.reset(seed=cfg.train.seed + episode)
    cosmos.update_obstacles(env.obstacles)
    cosmos.reset(env.positions)
    buffer.set_first_obs(obs, share_obs)
    
    ep_reward, ep_cost, ep_form_err, ep_min_dist, ep_collisions = 0.0, 0.0, 0.0, float('inf'), 0
    
    for step in range(cfg.env.max_steps):
        alphas, log_probs = mappo.get_actions(obs)
        values = mappo.get_values(share_obs)
        safe_actions = cosmos.project(alphas, env.positions, env.velocities, dt=cfg.env.dt)
        next_obs, next_share_obs, rewards, costs, dones, infos, _ = env.step(safe_actions)
        
        masks = (~dones).astype(np.float32).reshape(-1, 1)
        buffer.insert(next_obs, next_share_obs, alphas, log_probs, values, rewards, costs, masks)
        
        obs, share_obs = next_obs, next_share_obs
        total_steps += cfg.env.num_agents
        
        ep_reward += rewards[0, 0]
        ep_cost += costs[0, 0]
        ep_form_err += infos[0]["formation_error"]
        ep_min_dist = min(ep_min_dist, infos[0]["min_inter_dist"])
        ep_collisions += infos[0]["collisions"]
        
        if dones.all():
            break
    
    # PPO 更新
    last_values = mappo.get_values(share_obs)
    buffer.compute_returns_and_advantages(last_values)
    mappo.update(buffer)
    buffer.after_update()
    
    # 记录
    avg_form_err = ep_form_err / (step + 1)
    metrics['episode'].append(episode + 1)
    metrics['reward'].append(ep_reward)
    metrics['cost'].append(ep_cost)
    metrics['formation_error'].append(avg_form_err)
    metrics['min_dist'].append(ep_min_dist)
    metrics['collisions'].append(ep_collisions)
    
    # WandB 日志
    if USE_WANDB:
        wandb.log({
            "reward": ep_reward,
            "cost": ep_cost,
            "formation_error": avg_form_err,
            "min_inter_dist": ep_min_dist,
            "collisions": ep_collisions,
            "episode": episode + 1
        })
    
    # 打印进度
    if (episode + 1) % LOG_INTERVAL == 0:
        fps = total_steps / (time.time() - start_time)
        print(f"Ep {episode+1:4d}/{NUM_EPISODES} | R={ep_reward:7.2f} | Cost={ep_cost:4.0f} | "
              f"FormErr={avg_form_err:.4f} | MinDist={ep_min_dist:.3f} | Coll={ep_collisions:2d} | FPS={fps:.0f}")

elapsed = time.time() - start_time
print("=" * 70)
print(f"训练完成! 用时: {elapsed:.1f}s | 总碰撞: {sum(metrics['collisions'])}")

## 9. 训练曲线

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(15, 8))

axes[0, 0].plot(metrics['episode'], metrics['reward'], 'b-', alpha=0.7)
axes[0, 0].set_title('Episode Reward')
axes[0, 0].set_xlabel('Episode')
axes[0, 0].grid(True, alpha=0.3)

axes[0, 1].plot(metrics['episode'], metrics['cost'], 'r-', alpha=0.7)
axes[0, 1].set_title('Episode Cost')
axes[0, 1].set_xlabel('Episode')
axes[0, 1].grid(True, alpha=0.3)

axes[0, 2].plot(metrics['episode'], metrics['formation_error'], 'g-', alpha=0.7)
axes[0, 2].set_title('Formation Error')
axes[0, 2].set_xlabel('Episode')
axes[0, 2].grid(True, alpha=0.3)

axes[1, 0].plot(metrics['episode'], metrics['min_dist'], 'm-', alpha=0.7)
axes[1, 0].axhline(y=cfg.safety.safety_radius, color='r', linestyle='--', label='Safety Radius')
axes[1, 0].set_title('Min Inter-Agent Distance')
axes[1, 0].set_xlabel('Episode')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

axes[1, 1].plot(metrics['episode'], metrics['collisions'], 'c-', alpha=0.7)
axes[1, 1].set_title('Collisions per Episode')
axes[1, 1].set_xlabel('Episode')
axes[1, 1].grid(True, alpha=0.3)

# Smoothed reward
window = min(20, len(metrics['episode']) // 5) or 1
if window > 1:
    smoothed = np.convolve(metrics['reward'], np.ones(window)/window, mode='valid')
    axes[1, 2].plot(metrics['episode'][window-1:], smoothed, 'b-', linewidth=2)
else:
    axes[1, 2].plot(metrics['episode'], metrics['reward'], 'b-', linewidth=2)
axes[1, 2].set_title(f'Smoothed Reward (window={window})')
axes[1, 2].set_xlabel('Episode')
axes[1, 2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

if USE_WANDB:
    wandb.log({"training_curves": wandb.Image(fig)})

## 10. 评估与可视化

In [ ]:
# 评估一个 episode
obs, share_obs, _ = env.reset(seed=SEED + 1000)
cosmos.update_obstacles(env.obstacles)
cosmos.reset(env.positions)

trajectory = [env.positions.copy()]
obstacles_pos = env.obstacles.copy()
goal_pos = env.goal.copy()

for step in range(cfg.env.max_steps):
    alphas, _ = mappo.get_actions(obs, deterministic=True)
    safe_actions = cosmos.project(alphas, env.positions, env.velocities, dt=cfg.env.dt)
    obs, share_obs, _, _, dones, infos, _ = env.step(safe_actions)
    trajectory.append(env.positions.copy())
    if dones.all():
        break

trajectory = np.array(trajectory)
print(f"轨迹长度:     {len(trajectory)} 步")
print(f"最终编队误差: {infos[0]['formation_error']:.4f}")
print(f"碰撞次数:     {infos[0]['collisions']}")

In [ ]:
# 绘制轨迹
fig, ax = plt.subplots(figsize=(10, 10))
colors = plt.cm.tab10(np.linspace(0, 1, cfg.env.num_agents))

# 场地边界
rect = patches.Rectangle((-cfg.env.arena_size, -cfg.env.arena_size), 
                          2*cfg.env.arena_size, 2*cfg.env.arena_size,
                          linewidth=2, edgecolor='black', facecolor='none', linestyle='--')
ax.add_patch(rect)

# 障碍物
for obs in obstacles_pos:
    circle = patches.Circle((obs[0], obs[1]), obs[2], facecolor='gray', edgecolor='black', alpha=0.5)
    ax.add_patch(circle)

# 目标
ax.plot(goal_pos[0], goal_pos[1], 'r*', markersize=25, label='Goal')

# 轨迹
for i in range(cfg.env.num_agents):
    traj = trajectory[:, i, :]
    ax.plot(traj[:, 0], traj[:, 1], '-', color=colors[i], alpha=0.7, linewidth=2)
    ax.plot(traj[0, 0], traj[0, 1], 'o', color=colors[i], markersize=12, label=f'Agent {i}')
    ax.plot(traj[-1, 0], traj[-1, 1], 's', color=colors[i], markersize=12)

# 最终编队
final_pos = trajectory[-1]
for (i, j) in topology.edges():
    ax.plot([final_pos[i, 0], final_pos[j, 0]], [final_pos[i, 1], final_pos[j, 1]], 'k--', alpha=0.5)

ax.set_xlim(-cfg.env.arena_size * 1.1, cfg.env.arena_size * 1.1)
ax.set_ylim(-cfg.env.arena_size * 1.1, cfg.env.arena_size * 1.1)
ax.set_aspect('equal')
ax.set_title('COSMOS + RMPflow + MAPPO Formation Navigation', fontsize=14)
ax.legend(loc='upper right')
ax.grid(True, alpha=0.3)
plt.show()

if USE_WANDB:
    wandb.log({"trajectory": wandb.Image(fig)})

## 11. 动画

In [ ]:
%%capture
# 创建动画
fig, ax = plt.subplots(figsize=(8, 8))
colors = plt.cm.tab10(np.linspace(0, 1, cfg.env.num_agents))
T = len(trajectory)

def init():
    ax.clear()
    rect = patches.Rectangle((-cfg.env.arena_size, -cfg.env.arena_size), 
                              2*cfg.env.arena_size, 2*cfg.env.arena_size,
                              linewidth=2, edgecolor='black', facecolor='none', linestyle='--')
    ax.add_patch(rect)
    for obs in obstacles_pos:
        circle = patches.Circle((obs[0], obs[1]), obs[2], facecolor='gray', edgecolor='black', alpha=0.5)
        ax.add_patch(circle)
    ax.plot(goal_pos[0], goal_pos[1], 'r*', markersize=20)
    ax.set_xlim(-cfg.env.arena_size * 1.1, cfg.env.arena_size * 1.1)
    ax.set_ylim(-cfg.env.arena_size * 1.1, cfg.env.arena_size * 1.1)
    ax.set_aspect('equal')
    return []

def update(frame):
    ax.clear()
    init()
    pos = trajectory[frame]
    trail_start = max(0, frame - 30)
    for i in range(cfg.env.num_agents):
        trail = trajectory[trail_start:frame+1, i, :]
        ax.plot(trail[:, 0], trail[:, 1], '-', color=colors[i], alpha=0.5, linewidth=2)
        ax.plot(pos[i, 0], pos[i, 1], 'o', color=colors[i], markersize=12)
    for (i, j) in topology.edges():
        ax.plot([pos[i, 0], pos[j, 0]], [pos[i, 1], pos[j, 1]], 'k-', alpha=0.3)
    ax.set_title(f'Step {frame}/{T-1}')
    return []

frames = list(range(0, T, max(1, T // 100)))
anim = FuncAnimation(fig, update, init_func=init, frames=frames, blit=False, interval=50)

In [ ]:
# 显示动画
from IPython.display import HTML
HTML(anim.to_jshtml())

## 12. 安全统计

In [ ]:
total_collisions = sum(metrics['collisions'])
safe_episodes = sum(1 for c in metrics['collisions'] if c == 0)

print("=" * 50)
print("COSMOS 安全统计")
print("=" * 50)
print(f"总碰撞次数:       {total_collisions}")
print(f"零碰撞轮数:       {safe_episodes}/{NUM_EPISODES} ({100*safe_episodes/NUM_EPISODES:.1f}%)")
print(f"最终编队误差:     {metrics['formation_error'][-1]:.4f}")
print(f"最终最小距离:     {metrics['min_dist'][-1]:.3f}")
print(f"安全半径:         {cfg.safety.safety_radius}")
print("=" * 50)

if USE_WANDB:
    wandb.log({
        "total_collisions": total_collisions,
        "safe_episode_rate": safe_episodes / NUM_EPISODES,
        "final_formation_error": metrics['formation_error'][-1]
    })
    wandb.finish()
    print("\n✓ WandB 运行已完成")